## Gridplots and linked charts
In this notebook i will figure out some layout options for `bokeh` charts. The `gridplot()` function can be used to arrange Bokeh Plots in grid layout. Linking together various aspects of different plots can be a useful technique for data visualization. In Bokeh, such linkages are typically accomplished by sharing some plot component between plots.

### Gridplots
gridplot() allows to create a N x M layout for visualise several plots in an ordered way. It also collects all tools into a single toolbar, and the currently active tool is the same for all plots in the grid. **It is possible to leave “empty” spaces in the grid by passing None instead of a plot object.**

In [8]:
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.palettes import Viridis3
from bokeh.plotting import figure
import numpy as np

output_notebook()

Loading BokehJS ...

In [9]:
N = 100
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)
y2 = np.sin(x) + np.cos(x)

# create three plots
p1 = figure(plot_width=250, plot_height=250, title=None)
p1.circle(x, y0, size=3, color=Viridis3[0])

p2 = figure(plot_width=250, plot_height=250, title=None)
p2.circle(x, y1, size=3, color=Viridis3[1])

p3 = figure(plot_width=250, plot_height=250, title=None)
p3.circle(x, y2, size=3, color=Viridis3[2])

# make a grid
grid = gridplot([[p1, p2], [None, p3]])

# show the results
show(grid)

For convenience you can also just pass a list of plots, and specify the number of columns you want in your grid. For example,

`gridplot([[s1, s2], [s3, None]])`

and

`gridplot([s1, s2, s3], ncols=2)`

Furthermore, you can specify the size of the plots in the gridplot istruction.

In [10]:
# EXERCISE 1: create two plots in a gridplot, and link their ranges

In [11]:
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.palettes import Viridis3
from bokeh.plotting import figure

N = 100
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)
y2 = np.sin(x) + np.cos(x)

# create three plots
s1 = figure()
s1.circle(x, y0, size=3, color=Viridis3[0])
s2 = figure()
s2.circle(x, y1, size=3, color=Viridis3[1])
s3 = figure()
s3.circle(x, y2, size=3, color=Viridis3[2])

# make a grid
grid = gridplot([s1, s2, s3], ncols=2, plot_width=250, plot_height=250)

# show the results
show(grid)

### Linked panning and brushing

Linking together various aspects of different plots can be a useful technique for data visualization. In Bokeh, such linkages are typically accomplished by sharing some plot component between plots. Below is an example that demonstrates linked panning (where changing the range of one plot causes others to update) by sharing range objects between the plots. Some other things to look out for in this example:

- calling `figure()` multiple times to create multiple plots
- using `gridplot()` to arrange several plots in an array
- showing new glyphs using new glyph methods Figure.triangle and Figure.square
- hiding the toolbar by setting toolbar_location to None
- setting convenience arguments color (sets both line_color and fill_color) and alpha (sets both line_alpha and fill_alpha)


In [4]:
# prepare some data
N = 100
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)
y2 = np.sin(x) + np.cos(x)

# create a new plot
s1 = figure(title=None)
s1.circle(x, y0, size=3, color="navy", alpha=0.5)

# NEW: create a new plot and share both ranges
s2 = figure(x_range=s1.x_range, y_range=s1.y_range, title=None)
s2.circle(x, y1, size=3, color="firebrick", alpha=0.5)

# NEW: create a new plot and share only one range
s3 = figure(x_range=s1.x_range, title=None)
s3.circle(x, y2, size=3, color="olive", alpha=0.5)

# NEW: put the subplots in a gridplot
p = gridplot([[s1, s2, s3]], toolbar_location=None, plot_width=250, plot_height=250)

# show the results
show(p)

Another linkage that is often useful is linked brushing (where a selection on one plot causes a selection to update on other plots). Below is an example that demonstrates linked brushing by sharing a [ColumnDataSource](https://bokeh.pydata.org/en/latest/docs/reference/models/sources.html#bokeh.models.sources.ColumnDataSource) between two plots:

In [5]:
from bokeh.models import ColumnDataSource
# prepare some date
N = 300
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)

# NEW: create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=350, height=350, title=None)
left.circle('x', 'y0', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=350, height=350, title=None)
right.circle('x', 'y1', source=source)

# put the subplots in a gridplot
p = gridplot([[left, right]])

# show the results
show(p)

### Linked Selection

In [13]:
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

x = list(range(-20, 21))
y0 = [abs(xx) for xx in x]
y1 = [xx**2 for xx in x]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "box_select,lasso_select,help"

# create a new plot and add a renderer
left = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
left.circle('x', 'y0', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
right.circle('x', 'y1', source=source)

p = gridplot([[left, right]])

show(p)

In [14]:
# EXERCISE 2: create two plots in a gridplot, and link their data sources

### Linked selection with filtered data

In [15]:
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter

x = list(range(-20, 21))
y0 = [abs(xx) for xx in x]
y1 = [xx**2 for xx in x]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

# create a view of the source for one plot to use
view = CDSView(source=source, filters=[BooleanFilter([True if y > 250 or y < 100 else False for y in y1])])

TOOLS = "box_select,lasso_select"

# create a new plot and add a renderer
left = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
left.circle('x', 'y0', size=10, hover_color="firebrick", source=source)

# create another new plot, add a renderer that uses the view of the data source
right = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
right.circle('x', 'y1', size=10, hover_color="firebrick", source=source, view=view)

p = gridplot([[left, right]])

show(p)